# Projet d'optimisation 
## Par  Rania Fathi et Pierre Sion 

## I- Modélisation 

### Question 1 :
On définit l'élasticité-prix comme étant la variation relative de la quantité demandée par rapport au prix, soit encore: 
    $\varepsilon_i = \frac{\frac{dx_i}{x_i}}{\frac{dp_i}{p_i}} $ 
où l'on a noté $x_i$ la demande et $p_i$ le prix du ième produit 
    
Cela peut encore s'écrire: 
$\varepsilon_i = \frac{\partial x_i}{\partial p_i} \frac{p_i}{x_i}$

Comme on s'attend à ce que la demande pour un produit diminue si le prix augmente, on a $\varepsilon_i \leq 0$

Pour ce qui est de l'élasticité prix croisée entre le gouda et l'edam, elle vaut: 

$\varepsilon_c = \frac{\frac{dx_g}{x_g}}{\frac{dp_e}{p_e}} = \frac{\partial x_g}{\partial p_e} \frac{p_e}{x_g}$

Les biens considérés étant substituables, on a $\varepsilon_c \geq 0$ 

### Question 2 :
Le lait brut possède une quantité de matière grasse totale répartie entre les quatre produits, que l'on note $M$. En notant $m_i$ la teneur en matière grasse présente dans chaque produit, on a : 

$ \sum_{i=1}^{4}m_i x_i \leq M$

On fait par ailleurs le choix de mettre une inégalité plutôt qu'une égalité pour considérer le cas où tout le lait n'est pas transformé 

### Question 3 : 
De la même manière, on note $L$ la quantité totale de lactose contenue dans le lait, et $l_i$ celle contenue dans chaque produit, et on a alors la contrainte suivante: 

$\sum_{i=1}^{4} l_i x_i \leq L $

### Question 4 :

Afin de garantir la paix sociale, la moyenne (pondérée par la part de chaque produit dans le budget) des changements de prix relatifs ne doit pas être positive.
On note $b_i$ la part du ième produit dans le budget (déterminée à partir des données de l'année N-1). Cette contrainte s'écrit donc: 
$\sum_{i=1}^{4}b_i \frac{dp_i}{p_i} \leq 0$

Comme les contraintes dont on dispose jusque maintenant portent sur les quantités et non pas les prix, on la réécrit sous la forme suivante : $ \sum_{i=1}^4 \frac{b_i}{\epsilon_i} \frac{dx_i}{x_i} \leq 0$ 

### Question 5 :

_En supposant qu'on dispose déjà des $(p_i)_{1\leq i\leq4}$, $(b_i)_{1\leq i\leq4}$, $(\epsilon_i)_{1\leq i\leq4}$, $(m_i)_{1\leq i \leq 4}$, de $M$ et $L$_  
On dispose donc du problème d'optimisation sous contraintes suivant : $\max_{c(x) \leq 0} \Pi (x)$ 

où le profit $\Pi:\mathbf{R}^4 \rightarrow \mathbf{R}  $ vaut $\Pi(x) = \sum_{i=1}^4 p_i x_i $ (En toute rigueur, $\Pi$ est égal au revenu plutôt qu'au profit, mais l'omission du terme $-C$ correspondant au coût total du lait acheté ne modifie pas le problème d'optimisation à résoudre).

Et les contraintes $c : \mathbf{R}^4 \rightarrow \mathbf{R}^7$ sont les suivantes : 

$c_1(x) = - x_1 \\ 
c_2(x) = - x_2\\ 
c_3(x) = - x_3\\ 
c_4(x) = - x_4$

Ces quatre premières contraintes traduisent le fait que les quantités considérées sont nécessairement positives. Et nous disposons des deux contraintes des questions 2 et 3: 

$c_5(x) = \sum_{i=1}^{4}m_i x_i - M\\ 
c_6(x) = \sum_{i=1}^{4} l_i x_i - L $

Et enfin :
$c_7(x) =  \sum_{i=1}^4 \frac{b_i}{\epsilon_i} \frac{dx_i}{x_i} \leq 0$

NB: Il aurait été possible d'ajouter les contraintes portant sur le signe de l'élasticité des produits, imposant que l'augmentation du prix d'un produit réduit la demande, ou encore celles stipulant que les prix des produits se doivent d'être positifs 

On cherche donc à résoudre $max_{C^Tp-d \leq 0} b^T p - \frac{1}{2} p^T A p$ soit encore $min_{C^Tp-d \leq 0} \frac{1}{2} p^T A p- b^T p$

## II- Implémentation 

### Cadre de travail

In [24]:
import numpy as np 
from scipy import optimize
 #Données :
A = np.array([[3.0825,0,0,0],
              [0,0.0405,0,0],
              [0,0,0.0271,-0.0031],
              [0,0,-0.0031,0.0054]])
b = np.array([2671,135,103,19])

C = np.array([[-0.0401,-0.0162,-0.0039,0.0002],
              [-0.1326,-0.0004,-0.0034,0.0006],
              [1.5413,0,0,0],
              [0,0.0203,0,0],
              [0,0,0.0136,-0.0015],
              [0,0,-0.0016,0.0027],
              [0.0160,0.0004,0.0005,0.0002]])

d = np.array([-92.6,-29,2671,135,103,19,10])

x0=np.array([2055,54,63,17])
lambda0 = np.array([1e-2 for i in range(7)])

#Fonction à minimiser :
def f(x):
    return 0.5 * np.vdot(np.dot(x,A),x) - np.vdot(b,x)

def grad_f(x):
    return np.dot(A,x) - b 

# Contraintes:

def c(x) :
    return np.dot(C,x) - d 

def grad_c(x):
    return C

On commence par déterminer la solution de notre problème à l'aide de scipy, afin de pouvoir y comparer nos résultats: 

In [25]:
ineq_cons_f = {'type': 'ineq','fun' : c,'jac' : grad_c}
res_scipy_f = optimize.minimize(f, x0, method='SLSQP', jac = grad_f, constraints=[ineq_cons_f], options={'ftol': 1e-9, 'disp': True})
print("x_scipy_f : ", res_scipy_f.x)
print("c(x_scipy_f) : ", c(res_scipy_f.x))
print()

Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -214311.83304309193
            Iterations: 9
            Function evaluations: 5
            Gradient evaluations: 5
x_scipy_f :  [1732.95269029 1362.3463871   376.88087187 2156.24872887]
c(x_scipy_f) :  [-5.93232130e-09 -2.01322111e+02 -1.84613455e-05 -1.07344368e+02
 -1.01108793e+02 -1.37811378e+01  1.88918718e+01]



### I. Méthode d'Uzawa, à pas fixe et variable (wolfe): 

In [ ]:
def uzawa_fixed_step_array(fun, grad_fun, c, grad_c, x0, l, rho, lambda0, max_iter = 100000, epsilon_grad_L = 1e-8):
	k = 0
	xk = x0
	lambdak = lambda0
	grad_Lagrangienk_xk = grad_fun(xk) + np.dot(lambdak,grad_c(xk))
	while ((k<max_iter) and (np.linalg.norm(grad_Lagrangienk_xk)>epsilon_grad_L)):
		grad_Lagrangienk_xk = grad_fun(xk) + np.dot(lambdak,grad_c(xk))
		pk = -grad_Lagrangienk_xk
		xk = xk + l*pk;    
		lambdak= np.maximum(0, lambdak + rho*c(xk))	
		k = k + 1
	print("Nombre d'iterations : ", k)
	print("lambdak : ", lambdak)
	return xk


In [ ]:
def affichage(): 
    print("Uzawa fixed step V...")
    x_fixed_step_V = uzawa_fixed_step_array(f, grad_f, c, grad_c, x0, 0.0001, 0.1, lambda0)
    print(x_fixed_step_V) 

In [ ]:
def wolfe_step(fun, grad_fun, xk, pk, c1 = 0.25, c2 = 0.75, M = 1000):
	l_moins = 0
	l_plus = 0
	f_xk = fun(xk)
	grad_f_xk = grad_fun(xk)
	li = 0.0001
	i = 0
	while(i < M):
		if (fun(xk+li*pk)>(f_xk+c1*li*np.dot(grad_f_xk,pk))):
			l_plus = li
			li = (l_moins+l_plus)/2.0
		else:
			if (np.dot(grad_fun(xk+li*pk),pk) < c2*np.dot(grad_f_xk,pk)):
				l_moins = li
				if (l_plus == 0):
					li = 2*li
				else:
					li = (l_moins+l_plus)/2.0
			else:
				#print("Nb itérations : ", i)
				return li
		i = i + 1
	#print("Trop d'itérations de Wolfe")
	return li

def uzawa_wolfe_step_array(fun, grad_fun, c, grad_c, x0, rho, lambda0, max_iter = 100000, epsilon_grad_L = 1e-8):
	k = 0
	xk = x0
	lambdak = lambda0
	grad_Lagrangienk_xk = grad_fun(xk) + np.dot(lambdak,grad_c(xk))
	while ((k<max_iter) and (np.linalg.norm(grad_Lagrangienk_xk)>epsilon_grad_L)):
		Lagrangienk = lambda x : fun(x) + np.dot(lambdak, c(x))
		grad_Lagrangienk = lambda x : grad_fun(x) + np.dot(lambdak, grad_c(x))
		grad_Lagrangienk_xk = grad_Lagrangienk(xk)
		pk = -grad_Lagrangienk_xk
		lk = wolfe_step(Lagrangienk, grad_Lagrangienk, xk, pk)
		xk = xk + lk*pk;    
		lambdak = np.maximum(0, lambdak + rho*c(xk))
		k = k + 1
	print("Nombre d'iterations : ", k)
	print("lambdak_array : ", lambdak)
	return xk



In [ ]:
#Uzawa with gradient descent and Wolfe step do not converge (it is because of the initial step in wolfe_step() function. The choice 1 as initial step is good for Quasi-Newton or Newton methods, not for gradient descent. If you take li = 0.0001 instead of 1, the method will converge. You will see the convergence is similar to the gradient descent with fixed step.)
def affichage_wolfe():
    print("Uzawa wolfe step V...")
    x_wolfe_step_V = uzawa_wolfe_step_array(f, grad_f, c, grad_c, x0, 0.1, lambda0)
    print("x_wolfe_step_V : ", x_wolfe_step_V)
    print("g(x_wolfe_step_V) : ", c(x_wolfe_step_V))
    print()

### II. Méthode de Newton BFGS :

In [ ]:
def newton_BFGS_array(f, grad_f, c, grad_c, x0, lambda0, max_iter = 100000, epsilon_grad_L = 1e-5):
	k = 0
	xk = x0
	lambdak = lambda0
	Hk = np.identity(len(x0))
	grad_Lagrangienk_xk = grad_f(xk) + np.dot(lambdak,grad_c(xk))
	while ((k<max_iter) and (np.linalg.norm(grad_Lagrangienk_xk)>epsilon_grad_L)):
		Lagrangienk = lambda x : f(x) + np.dot(lambdak, c(x))
		grad_Lagrangienk = lambda x : grad_f(x) + np.dot(lambdak, grad_c(x))
		grad_Lagrangienk_xk = grad_Lagrangienk(xk)
		pk = -np.matmul(Hk,grad_Lagrangienk_xk)
		lk = wolfe_step(Lagrangienk, grad_Lagrangienk, xk, pk)
		xk1 = xk + lk*pk
		grad_Lagrangienk_xk1 = grad_Lagrangienk(xk1)
		sk = xk1 - xk
		yk = grad_Lagrangienk_xk1 - grad_Lagrangienk_xk
		gammak = 1.0/np.dot(yk, sk)
		Ak = np.identity(len(x0)) - gammak*np.multiply(sk[:, np.newaxis], yk)
		Bk = np.identity(len(x0)) - gammak*np.multiply(yk[:, np.newaxis], sk)
		Hk = np.matmul(np.matmul(Ak, Hk), Bk) + gammak*np.multiply(sk[:, np.newaxis], sk)
		xk = xk1 
		
		for i in range(len(c(xk))):
			rhok = np.dot(grad_c(xk)[i], np.matmul(Hk,grad_c(xk)[i]))
			lambdak[i] = np.maximum(0, lambdak[i] + (1/rhok)*c(xk)[i])
		k = k + 1
	print("Nombre d'iterations : ", k)
	print("lambdak : ", lambdak)
	return xk


In [ ]:
def affichage_BFGS():
    print("Newton BFGS V...")
    x_newton_BFGS_f = newton_BFGS_array(f, grad_f, c, grad_c, x0, lambda0)
    print("x_newton_BFGS_V : ", x_newton_BFGS_f)
    print("g(x_newton_BFGS_V) : ", c(x_newton_BFGS_f))
    print()

### Première méthode : Algorithme de Arrow-Hurwickz

On commence par construire le projecteur sur $\mathbb{R}_{+}$ :

In [7]:
def proj(x):
    for i in range(0,len(x)):
        if x[i]<0:
            x[i]=0
    return x

Ce qui permet d'implémenter l'algorithme AH : 

In [38]:
def algo_ah(x0,lambda0,epsilon=0.01,mu=10**-5,alpha=0.001, max_iterations = 10000, epsilon_frad_f = 1e-8):   
    lambdak=lambda0
    xl = x0
    xk = xl - epsilon*(grad_f(xl) + np.dot(C,lambdak))
    compteur = 1 
    while(compteur <= max_iterations and np.linalg.norm(grad_f(pk))>epsilon_grad_f):
        compteur += 1
        xk = xk - epsilon * (b - np.dot(A,pk) + (np.transpose(C)).dot(lambdak))
        lambdak = proj(lambdak + alpha * (np.dot(C,pk) - d))
        print(pk)
    return pk

In [40]:
print(algo_ah(x0, lambda0))

ValueError: shapes (7,4) and (7,) not aligned: 4 (dim 1) != 7 (dim 0)

In [37]:
grad_f(P0)

array([3663.5375, -132.813 , -101.3454,  -19.1035])